In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_column', 99)

/kaggle/input/kobe-bryant-shot-selection/sample_submission.csv.zip
/kaggle/input/kobe-bryant-shot-selection/data.csv.zip


In [2]:
sub = pd.read_csv('/kaggle/input/kobe-bryant-shot-selection/sample_submission.csv.zip')

In [3]:
data = pd.read_csv('/kaggle/input/kobe-bryant-shot-selection/data.csv.zip')

data.drop(['team_name','team_id','shot_id'],axis=1, inplace=True)
data['remaining'] = data['minutes_remaining'] * 60 + data['seconds_remaining']

data['loc_x_category'] = pd.cut(data.loc_x,bins=[data.loc_x.min()-1,-150,0,150,data.loc_x.max()+1], labels=['min_-150','-150_0','0_150','150_max'])
data['loc_y_category'] = pd.cut(data.loc_y,bins=[data.loc_y.min()-1,0,100,200,data.loc_y.max()+1], labels=['min_0','-0_100','100_200','200_max'])

data['basketDistance'] = np.sqrt(data['loc_x']*data['loc_x'] + data['loc_y']*data['loc_y'])
data['basketDistance_cat'] = pd.qcut(data.basketDistance, q=5, labels=['tooNear','Near','Mid','Far','tooFar'])

data.loc[ data['period']<5 ,'PlayingTime' ] = (data['period'] - 1 ) * 12 + (12 - data['minutes_remaining'])
data.loc[ data['period']>4 ,'PlayingTime' ] = 48 + (data['period'] - 5) * 5 + (5 - data['minutes_remaining'])

data.loc[data['PlayingTime']>48,'IsAddTime'] = True
data.loc[data['PlayingTime']<=48,'IsAddTime'] = False

#data['game_yearmon'] = data['game_date'].str.split('-').str[0] + data['game_date'].str.split('-').str[1]
#data['game_year'] = data['game_date'].str.split('-').str[0]
#data['game_month'] = data['game_date'].str.split('-').str[1]
#data['game_day'] = data['game_date'].str.split('-').str[2]

#data['season_time'] = pd.cut(data.game_month.astype('int'),bins=[0,3,6,13], labels=['mid_season','late_season','early_season'])
#data['player_life'] = pd.cut(data.game_year.astype('int'), bins=[1995,2003,2010,2017], labels=['rookie','prime','veteran'])

data.loc[data['matchup'].str.split().str[1] == '@','IsHome'] = False
data.loc[data['matchup'].str.split().str[1] != '@','IsHome'] = True

avg = data.groupby('combined_shot_type')['shot_distance'].transform('mean')
data['distanceDivPershot_type'] = data['shot_distance'] * avg

data.drop(['game_date','seconds_remaining'],axis=1,inplace=True)
data.drop(['game_event_id','game_id'],axis=1,inplace=True)

In [4]:
data['IsAddTime'] = data.IsAddTime.astype('bool')
data['IsHome'] = data.IsHome.astype('bool')
data['playoffs'] = data.playoffs.astype('bool')

category = data.columns[data.dtypes == 'object'].to_list()
numeric = data.drop(category,axis=1).columns.to_list()

data[category] = data[category].astype('category')

test = data[data['shot_made_flag'].isnull()]
train = data.dropna()

test.drop('shot_made_flag',axis=1,inplace=True)
y = train['shot_made_flag']
X = train.drop('shot_made_flag',axis=1)

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
from catboost import CatBoostClassifier

category = train.columns[train.dtypes == 'category'].to_list()

cat = CatBoostClassifier(random_state=42, learning_rate=0.01, n_estimators=2500,
                             eval_metric='Logloss',cat_features=category, verbose=100)

#cat.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100) # 6010
cat.fit(X,y)

0:	learn: 0.6915062	total: 122ms	remaining: 5m 5s
100:	learn: 0.6217365	total: 4.73s	remaining: 1m 52s
200:	learn: 0.6082982	total: 9.25s	remaining: 1m 45s
300:	learn: 0.6025846	total: 14.2s	remaining: 1m 43s
400:	learn: 0.5993683	total: 19.2s	remaining: 1m 40s
500:	learn: 0.5970011	total: 24.3s	remaining: 1m 37s
600:	learn: 0.5953234	total: 29.2s	remaining: 1m 32s
700:	learn: 0.5935978	total: 34.4s	remaining: 1m 28s
800:	learn: 0.5921853	total: 39.6s	remaining: 1m 23s
900:	learn: 0.5906835	total: 44.5s	remaining: 1m 19s
1000:	learn: 0.5891002	total: 49.6s	remaining: 1m 14s
1100:	learn: 0.5875651	total: 54.7s	remaining: 1m 9s
1200:	learn: 0.5860921	total: 59.7s	remaining: 1m 4s
1300:	learn: 0.5844235	total: 1m 4s	remaining: 59.9s
1400:	learn: 0.5828877	total: 1m 10s	remaining: 55.3s
1500:	learn: 0.5812979	total: 1m 15s	remaining: 50.4s
1600:	learn: 0.5797450	total: 1m 20s	remaining: 45.5s
1700:	learn: 0.5781644	total: 1m 26s	remaining: 40.5s
1800:	learn: 0.5768398	total: 1m 31s	remaini

In [6]:
from sklearn.metrics import log_loss

pred_cat = cat.predict(X_val)

print(log_loss(pred_cat,y_val)) # 11.289

10.806083632807303


In [7]:
from lightgbm import LGBMClassifier
# 6028
lgb = LGBMClassifier(random_state=42, learning_rate=0.01, n_estimators=400)
#lgb.fit(X_train, y_train, eval_set=[(X_val,y_val)],early_stopping_rounds=100, verbose=100,eval_metric='Logloss')
lgb.fit(X,y)

LGBMClassifier(learning_rate=0.01, n_estimators=400, random_state=42)

In [8]:
pred_lgb = lgb.predict(X_val)

print(log_loss(pred_lgb,y_val)) # 11.3740

10.61674926675552


In [9]:
from xgboost import XGBClassifier
# 6035
xgb = XGBClassifier(random_state=42, learning_rate=0.01, n_estimators=300,
                    enable_categorical=True,eval_metric=log_loss, tree_method='hist')
#xgb.fit(X_train,y_train, eval_set=[(X_val,y_val)],verbose=100,early_stopping_rounds=100)
xgb.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=True,
              eval_metric=<function log_loss at 0x78fa95fd3250>,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=42, ...)

In [10]:
pred_xgb = xgb.predict(X_val)

print(log_loss(pred_xgb,y_val)) #11.4442

10.735959793528863


In [11]:
sub1 = sub.copy()

pred = cat.predict_proba(test)[:,1]
sub1['shot_made_flag'] = pred
#sub1.to_csv('submission_cat.csv',index=False)

In [12]:
sub2 = sub.copy()

pred1 = lgb.predict_proba(test)[:,1]
sub2['shot_made_flag'] = pred1
#sub2.to_csv('submission_lgb.csv',index=False)

In [13]:
sub4 = sub.copy()

pred2 = xgb.predict_proba(test)[:,1]
sub4['shot_made_flag'] = pred2
#sub2.to_csv('submission_xgb.csv',index=False)

In [14]:
sub3 = sub.copy()

sub3['shot_made_flag'] = pred1 * 0.1 + pred * 0.8 + pred2 * 0.1
sub3.to_csv('submission_ensemble.csv',index=False)